In [1]:
# https://stackoverflow.com/questions/34621093/persist-elastic-search-data-in-docker-container
# TODOs: New KB upload with smaller text window
# TODOs: Common Crawl index upload
# TODOs: Good Logging

# import sys
# sys.path.append('../')

In [2]:
### CRAWL FILE-SYSTEM ###
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("KNOWLEDGE_BASE")

knowledge_path = "/Users/joshua.sheppard/wiki_extract_II"

def iter_filesys(path):
    if os.path.isfile(path):
        yield path

    elif os.path.isdir(path):
        for dir, _, filenames in os.walk(path):
                for f in filenames:
                    if not f.endswith('.DS_Store'):
                        yield os.path.join(dir, f)

    else:
        raise RuntimeError("Invalid path %s" % path)

kw_files = iter_filesys(knowledge_path)

kw_sample = []
for i in kw_files:
    kw_sample.append(i)

# print(kw_sample[:-3])
print(len(kw_sample))

17039


In [42]:
from multiprocessing.pool import ThreadPool as Pool
from tqdm import tqdm
from yake import KeywordExtractor
from summa import keywords

import json
import more_itertools
import re
import spacy

nlp = spacy.load("en_core_web_sm")
files = [f for f in kw_sample]

def get_contents_(filename):
    """Parse the contents of a file. Each line is a JSON encoded document."""
    documents = []

    with open(filename) as f:
        for line in f:
            doc = json.loads(line)

            if doc["text"] == "": continue
            if not doc: continue

            documents.append((doc['id'], doc["title"], doc["text"]))

    return documents


def generate_data():
        for file in files:
                #yield get_contents_2(file)
                docs = get_contents_(file)
                for doc in docs:
                    yield(doc)


def sentence_window(article, window=3, step=2): 
    """ Generates a list of sentences of sliding size = window """
    
    #sents = list(nlp(article).sents)
    # Low Cost using regex:
    sents = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article)
    
    if len(sents) == window:
        yield str(sents)

    for window in more_itertools.windowed(sents, n=window, step=2):
        yield window


def clean(passage):
    passage = str(passage)
    passage.encode("unicode_escape")
    passage.replace('"', '"')
    passage.strip()
    passage = re.sub("\n", "", passage)
    passage = re.sub('"', "'", passage)
    
    return passage

pool = Pool(8)
kw_extractor = KeywordExtractor(lan="en", n=3, top=5)

def passages(idx, source):
    count = 0
    for i in pool.apply(generate_data):
        count += 1
        id, title, article = i

        for window in sentence_window(article):
            passage = " ".join(clean(passage) for passage in window)

            if len(passage) < 50: continue
            else:

                yield {
                    "_index": idx,
                    "document": {
                        "id": id,
                        "source": source,
                        "title": title,
                        "text": passage,
                        "keyphrase": [i[0] for i in kw_extractor.extract_keywords(passage)],
                        #"keyphrase_summa": [i for i in keywords.keywords(passage).split("\n")]
                        }
                    }

In [43]:
### TEST KNOWLEDGE LOADER ###
test = []
trial = 1000
count = 0

with tqdm(total=trial) as pbar:
   for i in passages(idx="testing", source="wikipedia"):
      count += 1
      if count > trial:
        break
   
      test.append(i)
      pbar.update()


100%|██████████| 1000/1000 [00:03<00:00, 255.52it/s]


In [44]:
test

[{'_index': 'testing',
  'document': {'id': '61837831',
   'source': 'wikipedia',
   'title': 'Hedda Lundh',
   'text': 'Hedda Lundh (1921–2012) was a Danish journalist and schoolteacher who, under the German occupation of Denmark in World War II, was a Danish resistance fighter. Based at the time in Aarhus, she is remembered as a railway saboteur, explosives expert and courier in the resistance movement. Early life.',
   'keyphrase': ['Danish resistance fighter',
    'Denmark in World',
    'World War',
    'Hedda Lundh',
    'Danish journalist']}},
 {'_index': 'testing',
  'document': {'id': '61837831',
   'source': 'wikipedia',
   'title': 'Hedda Lundh',
   'text': "Early life. Born on 29 September 1921 in Korsør, Hedda Lundh was the daughter of the newspaper editor Theodor Lundh-Jensen (1884–1952) and Alpha Tusnelda Emilie Winckler (1887–1973). The youngest of three sisters, she was brought up in a middle-class home where her father called her his 'boy' as she climbed trees, joined

In [45]:
len(test)

1000

In [46]:
from elasticsearch import Elasticsearch

### DB CONFIG ###
PORT = "http://localhost:9200"
INDEX_WIKI = "knowledge"
SOURCE = "wikipedia"

errors_before_interrupt = 5
refresh_index_after_insert = False
max_insert_retries = 3
yield_ok = False

### INIT DB OBJECT ###
knowledge_base = Elasticsearch(
    PORT,
    retry_on_timeout=True
)

knowledge_base

<Elasticsearch(['http://localhost:9200'])>

In [ ]:
### REFRESH DB ###
#knowledge_base.delete_by_query(index=INDEX_NAME, query={"match_all": {}})

In [17]:
from elasticsearch.helpers import streaming_bulk, parallel_bulk
from tqdm.auto import tqdm
import spacy

### WRITE TO DB ###
errors_count = 0
chunk_size = 25000
counta = len(files)//chunk_size
successes = 0

with tqdm(total=counta) as pbar:
    for ok, result in parallel_bulk(knowledge_base, passages(idx=INDEX_WIKI, source=SOURCE), chunk_size=chunk_size, request_timeout=60*3):
        if ok is not True:
                logging.error('Failed to import data')
                logging.error(str(result))
                errors_count += 1

                if errors_count == errors_before_interrupt:
                    logging.fatal('Too many import errors, exiting with error code')
                    exit(1)

        successes += ok
        pbar.update()

  0%|          | 0/17039 [00:00<?, ?it/s]

INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:7.459s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:10.860s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:13.182s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:8.008s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:20.503s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:6.754s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:12.576s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:7.422s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:7.461s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:8.911s]
INFO:elastic_transport.transport:PUT http://localhost:92

ConnectionTimeout: Connection timed out

In [32]:
### CONSTRUCT NEWS DATASET ###

### LOAD ###
from datasets import load_dataset
cc_news = load_dataset("cc_news")

def iter_data():
    for i in cc_news["train"]:
        yield i

cc_news_data = iter_data()

### WRITE TO DISK ###
fout = open("../data/cc_news.jsonl", "w")
for ln in cc_news_data:
    fout.write(json.dumps({
        "title": ln["title"],
        "text": ln["text"],
        "domain": ln["domain"],
        "date": ln["date"],
        "url": ln["url"],
    }))
    
    fout.write("\n")

fout.close

Reusing dataset cc_news (/Users/joshua.sheppard/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6)


  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
test = []
trial = 10000
count = 0

for i in passages(idx="testing", source="news"):
    count += 1
    test.append(i)

    if count > trial:
        break

In [40]:
test[-5:]

[{'_index': 'testing',
  'document': {'title': 'AUTO RACING: Auto Racing Glance',
   'text': 'Track: Daytona International Speedway (oval, 2.5 miles). Race distance: 400 miles, 160 laps. Last year: Ricky Stenhouse Jr. won after starting sixth. Last race: Kyle Busch took the checkered flag at Chicagoland for the fifth time in 2018. Fast facts: Busch and Kevin Harvick each have five wins in their first 17 starts of the season.'}},
 {'_index': 'testing',
  'document': {'title': 'AUTO RACING: Auto Racing Glance',
   'text': "Last year: Ricky Stenhouse Jr. won after starting sixth. Last race: Kyle Busch took the checkered flag at Chicagoland for the fifth time in 2018. Fast facts: Busch and Kevin Harvick each have five wins in their first 17 starts of the season. That's only happened one other time since 1977, as Denny Hamlin and Jimmie Johnson had five victories by this point in the season in 2010. ...The last six races at Daytona have been determined by less than a half-second, and a 2007

In [44]:
### LOAD USING DB OBJECT ###

### DB CONFIG ###
INDEX_CC = "common_crawl_news"
SOURCE = "common-crawl"

INFO:elastic_db:Connecting to http://localhost:9200
INFO:elastic_db:Connecting to <Elasticsearch(['http://localhost:9200'])>


In [43]:
#self, index_name, source, iterator, chunk_size

knowledge_base.bulk_add(
    index_name=INDEX_CC,
    source=SOURCE,
    iterator=passages,
    chunk_size=10000
)

INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:4.573s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.934s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:7.892s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:9.578s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:9.720s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:3.766s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:3.874s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:4.443s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:2.980s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk [status:200 duration:3.460s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_

UnboundLocalError: local variable 'errors_count' referenced before assignment

In [ ]:
### TEST SEARCH ###
test_query = "government emails privacy"

def search(query_, db, index, k=5):
    results = db.search(
        index = db.elastic_index,
        query = {
            "size": k,
            "query": {
                "match": {
                    "document.text": query_,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    print(results)
    return (hits, doc_ids)

test = search(test_query, knowledge_base, INDEX_CC, k=2)[0][0]["_source"]["document"]["text"]
test

In [41]:
# from multiprocessing.pool import ThreadPool as Pool
# from tqdm import tqdm
# import more_itertools
# import re
#
# def clean(passage):
#     passage = str(passage)
#     passage.encode("unicode_escape")
#     passage.replace('"', '"')
#     passage.strip()
#     passage = re.sub("\n", "", passage)
#     passage = re.sub('"', "'", passage)
#
#     return passage
#
# def sentence_window(article, window=5, step=2):
#     """ Generates a list of sentences of sliding size = window """
#     sents = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article)
#
#     if len(sents) == window:
#         yield str(sents)
#
#     for window in more_itertools.windowed(sents, n=window, step=2):
#         yield window
#
# pool = Pool(8)
# def passages(idx, source):
#     count = 0
#     # with tqdm(total=len(files)) as pbar:
#     for i in pool.apply(iter_data):
#         count += 1
#         article = i["text"]
#
#         for window in sentence_window(article):
#             passage = " ".join(clean(passage) for passage in window)
#
#             #if len(passage) < 50: continue
#
#             yield {
#                 "_index": idx,
#                 "document": {
#                     "source": source,
#                     "title": i["title"],
#                     "text": passage
#                     }
#                 }
#
#         # pbar.update()
#

In [ ]:
# import re
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk, parallel_bulk
# from tqdm.auto import tqdm
# import spacy

# es = wiki_ev
# errors_count = 0

# # TODOs: Tune chunk size
# chunk_size = 25000
# counta = len(files)//chunk_size
# successes = 0

# # with tqdm(total=counta) as pbar:
# for ok, result in parallel_bulk(es, passages(idx=INDEX_NAME, source=SOURCE), chunk_size=chunk_size, request_timeout=60*3):

# # for ok, result in tqdm(streaming_bulk(es, index=INDEX_NAME, actions=passages(idx=INDEX_NAME, source=SOURCE), 
# #                         chunk_size=chunk_size, request_timeout=60*3, max_retries=3)):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1

#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)
    
#     successes += ok

In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk, parallel_bulk
# from tqdm import tqdm

# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."

# "https://github.com/elastic/elasticsearch-py/issues/297"

# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}

#         yield {
#             "_index": idx_,
#             "document": {
#                 "id": idx,
#                 "title": title,
#                 "text": text
#             }
#         }

# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# errors_count = 0

# # TODOs: Increase Chunk Size, with extended Timeout + handeling
# # TODOs: Experiment-Check with Yield OK
# for ok, result in parallel_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1

#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:

# import spacy

# nlp = spacy.load("en_core_web_sm")

# def sentence_window(article, window=3, step=2): 
#     """ Generates a list of sentences of sliding size = window """
    
#     sents = list(nlp(article).sents)
    
#     if len(sents) == window:
#         yield sents

#     for i in range(0, len(sents)):
#         yield(sents[i:i + window])

# def load_data(docs, idx_):
#     """ Generates an evidence document to be inserted into ES Index """
#     for doc in docs:
#         idx, title, text = doc

#         for paragraph in sentence_window(text):
#             yield {
#                 "_index": idx_,
#                 "document": {
#                     "id": idx,
#                     "title": title,
#                     "text": paragraph
#                 }
#             }


In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk
# from tqdm import tqdm
#
# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."
#
# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}
#
#         yield {
#             "_index": idx_,
#             "document": doc
#         }
#
# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# #helpers.bulk(es, load_data(wiki_data, "wiki_evidence"), raise_on_error=False, chunk_size=500)
# errors_count = 0
# # TODOs: Increase Chunk Size, with extended Timeout + handelling
# for ok, result in streaming_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3, yield_ok=yield_ok, refresh=refresh_index_after_insert):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1
#
#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:
# for article in wiki_data:
#     _id, title, text = article
#     doc = {"id": _id, "title": title, "text": text}
#
#     wiki_ev.add_doc(doc)

In [ ]:
# https://github.com/elastic/elasticsearch-py/issues/297

In [ ]:
### DOCUMENT IMPORT: FULL-TEXTS ###
# from multiprocessing import Pool
# # from utils import get_contents
# import utils
# from tqdm import tqdm
# import json

# # TODOs: USE A GENERATOR OBJECT
# import spacy
# nlp = spacy.load("en_core_web_sm")
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# count = 0
# test = []
# with tqdm(total=len(files)) as pbar:
#     for documents in p.map(utils.get_contents_2, files):
#             for doc in documents:
#                 _id, title, text = doc
#
#             # count += 1
#             # doc_ = {"id": _id, "title": title, "text": text}
#
#             #wiki_ev.add_doc(doc)
#             test.append(doc_)
#     pbar.update()

In [ ]:
# from elasticsearch import Elasticsearch

# # INIT OBJECT

# # TODOs: Persist a Generator Object
# PORT = "http://localhost:9200"
# INDEX_NAME = "wiki_evidence"
# errors_before_interrupt = 5
# refresh_index_after_insert = False
# max_insert_retries = 3
# yield_ok = False

# wiki_ev = Elasticsearch(
#     PORT,
#     #http_auth=(es_api_user, es_api_password)
#     retry_on_timeout=True,  # should timeout trigger a retry on different node?
# )

# wiki_ev.elastic_index = INDEX_NAME

# wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [ ]:
# ### DOCUMENT IMPORT: SEGMENTED-TEXTS ###

# from multiprocessing import Pool
# # from utils import get_contents
# from tqdm import tqdm

# # p = Pool(8)
# # files = [f for f in kw_sample]
# #
# # count = 0
# # test = []
# # with tqdm(total=len(files)) as pbar:
# #     for documents in p.map(get_contents, files):
# #         for doc in documents:
# #             _id, title, text = doc
# #
# #             count += 1
# #             doc_ = {"id": _id, "title": title, "text": text}
# #
# #             #wiki_ev.add_doc(doc)
# #             test.append(doc)
# #     pbar.update()

In [ ]:
# ### QUERY DB ###
# import elastic_db
# # from elastic_db import ElasticDB
# #
# # # Params
# # PORT = "http://localhost:9200"
# # INDEX = "wiki_evidence"
# # DOC = "evidence"
# #
# # # Init Elasticsearch DB
# # wiki_ev_ = ElasticDB(elastic_port=PORT, elastic_index=INDEX, elastic_doc=DOC)
# #
# # results = wiki_ev_.search("exploitation a wider public debate indecency adult")
# # results

In [ ]:
### SQLITE LOAD ###
# from multiprocessing import Pool
# import utils
# from tqdm import tqdm
# import sqlite3

# import spacy
# import uuid

# nlp = spacy.load("en_core_web_sm")

# def paragraphs(document):
#     start = 0
#     document = nlp(document)
#     passages = []
#     for token in document:
#         if token.is_space and token.text.count("\n") > 1:
#             yield document[start:token.i]
#             start = token.i
#     yield document[start:]


# def get_contents(filename):
#     """Parse the contents of a file. Each line is a JSON encoded document."""
#     documents = []

#     with open(filename) as f:
#         for line in f:
#             doc = json.loads(line)

#             if doc["text"] == "": continue
#             if not doc: continue

#             passages = [str(i) for i in paragraphs(doc["text"])][0].split("\n")

#             for passage in passages:
#                 if len(passage) < 50:
#                     continue

#                 documents.append((str(uuid.uuid4()).replace('-',''), doc['id'], doc["title"], passage))

#     return documents

# save_path = "../data/wiki_evidence.db"
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# conn = sqlite3.connect(save_path)
# c = conn.cursor()
#
# documents = "documents"
# c.execute(f"CREATE TABLE documents (id PRIMARY KEY, id_, title, text);")
#
# count = 0
# step = 100
# batches = [files[i:i + step] for i in range(0, len(files), step)]
#
# for i, batch in enumerate(batches):
#     logger.info(f"[.... Batch #{i} .....]")
#     with tqdm(total=len(batch)) as pbar:
#         for document in tqdm(p.imap_unordered(get_contents, files)):
#             count += 1
#             for content in document:
#                 # _id, title, passage = content
#                 c.executemany("INSERT INTO documents VALUES (?,?,?,?)", (content,))
#
#         pbar.update()
#         logger.info(f"[Uploaded {count} documents]")
#
# conn.commit()
# conn.close()

In [ ]:
# seq = [0, 1, 2, 3, 4, 5]
# window_size = 3
# step = 2

# # steps = 0, 2, 4 

# for i in range(0, len(seq) - window_size + 1, step):
#     print(i)
#     # print(i + window_size)
#     if i + window_size > len(seq):
#         # print(window_size)
#         window_size = i + window_size - len(seq)
    
#     print(seq[i: i + window_size])